In [1]:
import surprise as sp
import pandas as pd
import numpy as np
import math
from collections import defaultdict
from scipy import stats

#1.Loading the dataset and Excluding the outliers
kbdata = pd.read_csv("/data/fjsdata/ctKngBase/kb.csv", sep='|', low_memory=False)
kbdata = kbdata.loc[(kbdata['num']<200)]#seven months, one per day
print ('Dataset shape is:%d rows and %d columns'%(kbdata.shape[0],kbdata.shape[1]))

#2.Calculating the mean of CSRs and KEs.
Ab_csr = kbdata['num'].groupby(kbdata['csr']).mean()#the type of groupby is Series
Di_ke = kbdata['num'].groupby(kbdata['ke']).mean()
#3.Calculating the IRT of every pair<csr,ke> which is the pdf of norm
#mu=Di_ke,sigma=Ab_csr
#x[2]=num,x[1]=ke,x[0]=csr
kbdata['irt'] = kbdata.apply(lambda x: "{:.8f}".format(stats.norm.pdf(int(x[2]), Di_ke[int(x[1])], Ab_csr[int(x[0])])),axis=1)
#kbdata['irt'] = kbdata['irt'].apply(lambda x: 1 if float(x)>0 else 0)
print ('Number of 0:')
print (kbdata['irt'].value_counts())
print (kbdata[kbdata['irt']==0].shape)

Dataset shape is:2547452 rows and 3 columns
Number of 0:
0.00000000    94453
0.00000001     4162
0.00000002     2091
0.00000003     1447
0.00000004     1057
0.06582965     1047
0.00000005      918
0.16498579      776
0.15383277      744
0.00000006      721
0.13034388      670
0.11983165      663
0.00000007      654
0.16067967      612
0.12443317      583
0.14342331      582
0.00000008      576
0.00000009      505
0.00000010      499
0.00000011      439
0.10575668      432
0.16019196      405
0.00000012      398
0.17554315      395
0.19255729      394
0.00000013      380
0.15427646      376
0.12000412      370
0.14850679      367
0.15242517      366
              ...  
0.22198899        1
0.15051099        1
0.06069347        1
0.01952361        1
0.05275973        1
0.11683059        1
0.11746221        1
0.07722803        1
0.09594550        1
0.10126178        1
0.05624077        1
0.04826663        1
0.08494356        1
0.01645989        1
0.05035707        1
0.07638198        1
0.0

In [2]:
#NMF,threshold=0.1
#2.Transforming into data format of surprise and spliting the train-set and test-set
# The columns must correspond to user id, item id and ratings (in that order).
reader = sp.Reader(rating_scale=(0, 1))
spdata = sp.Dataset.load_from_df(kbdata[['csr', 'ke', 'irt']],reader)
# sampling random trainset and testset, and test set is made of 10% of the ratings.
#trainset, testset = sp.model_selection.train_test_split(spdata, test_size=.1)
trainset = spdata.build_full_trainset()
testset = trainset.build_testset()

#3.Training the model and predicting ratings for the testset
algo = sp.NMF()
algo.fit(trainset)
predictions = algo.test(testset)#testset include positive and negtive sample.

#4.measuring the performance of SVD by RMSE
print (sp.accuracy.rmse(predictions))

RMSE: 0.0586
0.058622259565876765


In [6]:
#SVD,threshold=0.1
#2.Transforming into data format of surprise and spliting the train-set and test-set
# The columns must correspond to user id, item id and ratings (in that order).
reader = sp.Reader(rating_scale=(0, 1))
spdata = sp.Dataset.load_from_df(kbdata[['csr', 'ke', 'irt']],reader)
# sampling random trainset and testset, and test set is made of 10% of the ratings.
#trainset, testset = sp.model_selection.train_test_split(spdata, test_size=.1)
trainset = spdata.build_full_trainset()
testset = trainset.build_testset()

#3.Training the model and predicting ratings for the testset
algo = sp.SVD()
algo.fit(trainset)
predictions = algo.test(testset)#testset include positive and negtive sample.

#4.measuring the performance of SVD by RMSE
print (sp.accuracy.rmse(predictions))

RMSE: 0.0334
0.033445300632199186


In [7]:
#NMF,threshold=0.1
#2.Transforming into data format of surprise and spliting the train-set and test-set
# The columns must correspond to user id, item id and ratings (in that order).
reader = sp.Reader(rating_scale=(0, 1))
spdata = sp.Dataset.load_from_df(kbdata[['csr', 'ke', 'irt']],reader)
# sampling random trainset and testset, and test set is made of 10% of the ratings.
#trainset, testset = sp.model_selection.train_test_split(spdata, test_size=.1)
trainset = spdata.build_full_trainset()
testset = trainset.build_testset()

#3.Training the model and predicting ratings for the testset
algo = sp.NMF()
algo.fit(trainset)
predictions = algo.test(testset)#testset include positive and negtive sample.

#4.measuring the performance of SVD by RMSE
print (sp.accuracy.rmse(predictions))

RMSE: 0.0586
0.05861593588371108


In [10]:
NUM_USERS = kbdata['csr'].max() + 1
NUM_ITEMS = kbdata['ke'].max() + 1
print (NUM_USERS)
print (NUM_ITEMS)
print (NUM_USERS*NUM_ITEMS)
print (len(kbdata))
print('dataset density:{:f}'.format((len(kbdata)-94453)*1.0/(NUM_USERS*NUM_ITEMS)))

10216
96324
984045984
2547452
dataset density:0.002493


In [2]:
import surprise as sp
import pandas as pd
import numpy as np
import math
from collections import defaultdict
from scipy import stats

#1.Loading the dataset and Excluding the outliers
kbdata = pd.read_csv("/data/fjsdata/ctKngBase/kb.csv", sep='|', low_memory=False)
kbdata = kbdata.loc[(kbdata['num']<200)]#seven months, one per day
print ('Dataset shape is:%d rows and %d columns'%(kbdata.shape[0],kbdata.shape[1]))

#2.Calculating the mean of CSRs and KEs.
Ab_csr = kbdata['num'].groupby(kbdata['csr']).mean()#the type of groupby is Series
Di_ke = kbdata['num'].groupby(kbdata['ke']).mean()
#3.Calculating the IRT of every pair<csr,ke> which is the pdf of norm
#mu=Di_ke,sigma=Ab_csr
#x[2]=num,x[1]=ke,x[0]=csr
kbdata['irt'] = kbdata.apply(lambda x: "{:.8f}".format(stats.norm.pdf(int(x[2]), Di_ke[int(x[1])], Ab_csr[int(x[0])])),axis=1)

#SVD,threshold=0.1
#2.Transforming into data format of surprise and spliting the train-set and test-set
# The columns must correspond to user id, item id and ratings (in that order).
reader = sp.Reader(rating_scale=(0, 1))
spdata = sp.Dataset.load_from_df(kbdata[['csr', 'ke', 'irt']],reader)
# sampling random trainset and testset, and test set is made of 10% of the ratings.
#trainset, testset = sp.model_selection.train_test_split(spdata, test_size=.1)
trainset = spdata.build_full_trainset()
testset = trainset.build_testset()

#3.Training the model and predicting ratings for the testset
algo = sp.SVD(n_factors=20,lr_all=3e-5,reg_all=0.02)
algo.fit(trainset)
predictions = algo.test(testset)#testset include positive and negtive sample.

#4.measuring the performance of SVD by RMSE
print (sp.accuracy.rmse(predictions))

Dataset shape is:2547452 rows and 3 columns
RMSE: 0.0605
0.06050782172230934


In [3]:
import surprise as sp
import pandas as pd
import numpy as np
import math
from collections import defaultdict
from scipy import stats

#1.Loading the dataset and Excluding the outliers
kbdata = pd.read_csv("/data/fjsdata/ctKngBase/kb.csv", sep='|', low_memory=False)
kbdata = kbdata.loc[(kbdata['num']<200)]#seven months, one per day
print ('Dataset shape is:%d rows and %d columns'%(kbdata.shape[0],kbdata.shape[1]))

#2.Calculating the mean of CSRs and KEs.
Ab_csr = kbdata['num'].groupby(kbdata['csr']).mean()#the type of groupby is Series
Di_ke = kbdata['num'].groupby(kbdata['ke']).mean()
#3.Calculating the IRT of every pair<csr,ke> which is the pdf of norm
#mu=Di_ke,sigma=Ab_csr
#x[2]=num,x[1]=ke,x[0]=csr
kbdata['irt'] = kbdata.apply(lambda x: "{:.8f}".format(stats.norm.pdf(int(x[2]), Di_ke[int(x[1])], Ab_csr[int(x[0])])),axis=1)

#SVD,threshold=0.1
#2.Transforming into data format of surprise and spliting the train-set and test-set
# The columns must correspond to user id, item id and ratings (in that order).
reader = sp.Reader(rating_scale=(0, 1))
spdata = sp.Dataset.load_from_df(kbdata[['csr', 'ke', 'irt']],reader)
# sampling random trainset and testset, and test set is made of 10% of the ratings.
#trainset, testset = sp.model_selection.train_test_split(spdata, test_size=.1)
trainset = spdata.build_full_trainset()
testset = trainset.build_testset()

#3.Training the model and predicting ratings for the testset
algo = sp.SVD(n_factors=50,lr_all=3e-5,reg_all=0.02)
algo.fit(trainset)
predictions = algo.test(testset)#testset include positive and negtive sample.

#4.measuring the performance of SVD by RMSE
print (sp.accuracy.rmse(predictions))

Dataset shape is:2547452 rows and 3 columns
RMSE: 0.0749
0.07489168018822281


In [4]:
import surprise as sp
import pandas as pd
import numpy as np
import math
from collections import defaultdict
from scipy import stats

#1.Loading the dataset and Excluding the outliers
kbdata = pd.read_csv("/data/fjsdata/ctKngBase/kb.csv", sep='|', low_memory=False)
kbdata = kbdata.loc[(kbdata['num']<200)]#seven months, one per day
print ('Dataset shape is:%d rows and %d columns'%(kbdata.shape[0],kbdata.shape[1]))

#2.Calculating the mean of CSRs and KEs.
Ab_csr = kbdata['num'].groupby(kbdata['csr']).mean()#the type of groupby is Series
Di_ke = kbdata['num'].groupby(kbdata['ke']).mean()
#3.Calculating the IRT of every pair<csr,ke> which is the pdf of norm
#mu=Di_ke,sigma=Ab_csr
#x[2]=num,x[1]=ke,x[0]=csr
kbdata['irt'] = kbdata.apply(lambda x: "{:.8f}".format(stats.norm.pdf(int(x[2]), Di_ke[int(x[1])], Ab_csr[int(x[0])])),axis=1)

#SVD,threshold=0.1
#2.Transforming into data format of surprise and spliting the train-set and test-set
# The columns must correspond to user id, item id and ratings (in that order).
reader = sp.Reader(rating_scale=(0, 1))
spdata = sp.Dataset.load_from_df(kbdata[['csr', 'ke', 'irt']],reader)
# sampling random trainset and testset, and test set is made of 10% of the ratings.
#trainset, testset = sp.model_selection.train_test_split(spdata, test_size=.1)
trainset = spdata.build_full_trainset()
testset = trainset.build_testset()

#3.Training the model and predicting ratings for the testset
algo = sp.SVD(n_factors=200,lr_all=3e-5,reg_all=0.02)
algo.fit(trainset)
predictions = algo.test(testset)#testset include positive and negtive sample.

#4.measuring the performance of SVD by RMSE
print (sp.accuracy.rmse(predictions))

Dataset shape is:2547452 rows and 3 columns
RMSE: 0.1153
0.11531762201920222


In [1]:
import surprise as sp
import pandas as pd
import numpy as np
import math
from collections import defaultdict
from scipy import stats

#1.Loading the dataset and Excluding the outliers
kbdata = pd.read_csv("/data/fjsdata/ctKngBase/kb.csv", sep='|', low_memory=False)
#seven month multiply thirty days per month is equal to 210,and one time per day multiply 210 is 210.
kbdata = kbdata.loc[(kbdata['num']<200)]
print ('Dataset shape is:%d rows and %d columns'%(kbdata.shape[0],kbdata.shape[1]))

#2.Calculating the mean of CSRs and KEs.
Ab_csr = kbdata['num'].groupby(kbdata['csr']).mean()#the type of groupby is Series
Di_ke = kbdata['num'].groupby(kbdata['ke']).mean()
#3.Calculating the IRT of every pair<csr,ke> which is the pdf of norm
#mu=Di_ke,sigma=Ab_csr
#x[2]=num,x[1]=ke,x[0]=csr
kbdata['irt'] = kbdata.apply(lambda x: "{:.8f}".format(stats.norm.pdf(int(x[2]), Di_ke[int(x[1])], Ab_csr[int(x[0])])),axis=1)
#2.Transforming into data format of surprise and spliting the train-set and test-set
# The columns must correspond to user id, item id and ratings (in that order).
reader = sp.Reader(rating_scale=(0, 1))
spdata = sp.Dataset.load_from_df(kbdata[['csr', 'ke', 'irt']],reader)
# sampling random trainset and testset, and test set is made of 10% of the ratings.
#trainset, testset = sp.model_selection.train_test_split(spdata, test_size=.1)
trainset = spdata.build_full_trainset()
testset = trainset.build_testset()

#3.Training the model and predicting ratings for the testset
print ("%3s%20s" % ('K','RMSE'))
for k in [50,100,200,500]:
    #3.Training the model and predicting ratings for the testset
    algo = sp.SVD(n_factors=k)
    algo.fit(trainset)
    predictions = algo.test(testset)#testset include positive and negtive sample.

    #4.rating prediction
    RMSE = sp.accuracy.rmse(predictions)
    print ("%3s%20.8f" % (k, RMSE))

Dataset shape is:2547452 rows and 3 columns
  K                RMSE
RMSE: 0.0315
 50          0.03152883
RMSE: 0.0334
100          0.03338732
RMSE: 0.0354
200          0.03541587
RMSE: 0.0351
500          0.03508189
